In [1]:
from pathlib import Path
import sys

# Define the base repository directory
repo_dir = Path.home() / "Desktop" / "eye_ai" / "Github"  # Update to your GitHub repo location

# Update the load path so Python can find modules for the model
sys.path.insert(0, str(repo_dir / "deriva-ml"))
sys.path.insert(0, str(repo_dir / "eye-ai-ml"))

# Reload extensions if needed
%load_ext autoreload
%autoreload 2


In [2]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI
from deriva_ml.dataset_bag import DatasetBag
import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva_ml.deriva_ml_base import MLVocab as vc
from deriva_ml.execution_configuration import ExecutionConfiguration, Workflow, Execution

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [3]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
host = 'dev.eye-ai.org'
catalog_id = "428"

# host = 'www.eye-ai.org'
# catalog_id = "21"
# catalog_id = "eye-ai"

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2024-11-22 10:34:15,643 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-11-22 10:34:15,645 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


You are already logged in.


In [4]:
# Variables to configure the rest of the notebook.
cache_dir = Path.home() / 'Desktop/test_cache'
working_dir = Path.home() / 'Desktop/test_cache'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2024-11-22 10:34:15,679 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-11-22 10:34:15,680 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-11-22 10:34:17,928 - INFO - Loading dirty model.  Consider commiting and tagging: 1.1.0.post76+git.403fdd90.dirty


# 1. Configuration

In [5]:
# Add Workflow Vocab terms
EA.add_term(vc.workflow_type, "Test Workflow", description="A test Workflow for new DM")
# Workflow instance
test_workflow = Workflow(
    name="Test New Workflow-multimodal",
    url="https://github.com/informatics-isi-edu/eye-ai-exec/blob/Data_Model_Update/notebooks/schema_changes/template_multimodal.ipynb",
    workflow_type="Test Workflow"
)
# Configuration instance
config = ExecutionConfiguration(
    bdbags=['2-AGAW'],
    models = ['2-4JR6'],
    execution=Execution(description="Sample Test Execution"),
    workflow=test_workflow,
    description="Our Test Workflow instance")
# Initialize execution
configuration_record = EA.initialize_execution(config)
execution_rid = configuration_record.execution_rid


2024-11-22 10:34:25,652 - INFO - Configuration validation successful!
2024-11-22 10:34:28,090 - INFO - Initializing downloader: GenericDownloader v1.7.4 [Python 3.10.15, macOS-10.16-x86_64-i386-64bit]
2024-11-22 10:34:28,098 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-11-22 10:34:28,101 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-11-22 10:34:28,112 - INFO - Validating credentials for host: dev.eye-ai.org
2024-11-22 10:34:28,269 - INFO - Creating bag directory: /var/folders/f8/85_209cx72l8mnt3km_8dx540000gn/T/tmp3z8eeumm/Dataset_2-AGAW
2024-11-22 10:34:28,272 - INFO - Creating bag for directory /var/folders/f8/85_209cx72l8mnt3km_8dx540000gn/T/tmp3z8eeumm/Dataset_2-AGAW
2024-11-22 10:34:28,276 - INFO - Creating data directory
2024-11-22 10:34:28,278 - INFO - Moving /private

In [6]:
print(configuration_record)
configuration_record.bag_paths[0]

caching_dir: /Users/zhiweili/Desktop/test_cache
working_dir: /Users/zhiweili/Desktop/test_cache/zhiweili/EyeAI_working
execution_rid: 5-R6G4
workflow_rid: 5-R6FW
bag_paths: [PosixPath('/Users/zhiweili/Desktop/test_cache/2-AGAW_fdfe65b9206c3db5327d1d9da43573a14f12610c5ff14b8c4dd7a02f68207d30/Dataset_2-AGAW')]
asset_paths: [PosixPath('/Users/zhiweili/Desktop/test_cache/zhiweili/EyeAI_working/5-R6G4/models/optic_disk_crop_model.hdf5')]
configuration: bdbags=['2-AGAW'] models=['2-4JR6'] workflow=Workflow(name='Test New Workflow-multimodal', url='https://github.com/informatics-isi-edu/eye-ai-exec/blob/Data_Model_Update/notebooks/schema_changes/template_multimodal.ipynb', workflow_type='Test Workflow', version=None, description=None) execution=Execution(description='Sample Test Execution') description='Our Test Workflow instance'


PosixPath('/Users/zhiweili/Desktop/test_cache/2-AGAW_fdfe65b9206c3db5327d1d9da43573a14f12610c5ff14b8c4dd7a02f68207d30/Dataset_2-AGAW')

In [18]:
ds_bag = DatasetBag(configuration_record.bag_paths[0])

# Test modality extraction

In [22]:
subject = ds_bag.get_table_as_dataframe('Subject')
observation = ds_bag.get_table_as_dataframe('Observation')
image = ds_bag.get_table_as_dataframe('Image')
clinic = ds_bag.get_table_as_dataframe('Clinical_Records')
report = ds_bag.get_table_as_dataframe('Report')
RNFL_OCR = ds_bag.get_table_as_dataframe('OCR_RNFL')


,RID,RCT,RMT,RCB,RMB,Subject_ID,Subject_Gender,Subject_Ethnicity
0,2-7KTR,2024-04-15 23:44:22.184165+00,2024-10-18 18:01:12.957676+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,Multimodal_1,F,Caucasian
1,2-7KTT,2024-04-15 23:44:22.184165+00,2024-10-18 18:01:12.957676+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,Multimodal_2,M,Caucasian
2,2-7KTW,2024-04-15 23:44:22.184165+00,2024-10-18 18:01:12.957676+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,Multimodal_3,M,African Descent
3,2-7KTY,2024-04-15 23:44:22.184165+00,2024-10-18 18:01:12.957676+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,Multimodal_4,F,Latin American
4,2-7KV0,2024-04-15 23:44:22.184165+00,2024-10-18 18:01:12.957676+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,Multimodal_5,M,African Descent


In [30]:
image

,RID,RCT,RMT,RCB,RMB,URL,Filename,Description,Length,MD5,Observation,Image,Source_Image,Process,Image_Side,Image_Angle,Image_Tag
0,2-C2SR,2024-05-30 09:19:13.08829+00,2024-10-18 17:58:23.664889+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,/hatrac/images/multimodal/scans/subject/2/obse...,data/assets/Image/2-660-IMAGE_0_(33).jpg,,6067012,8d01bcdf8b5e70b9c05848c9a60718a9,2-7RGW,,,,Unknown,Unknown,


In [29]:
# data_path = configuration_record.bag_paths[0]
# subject_path = data_path/ 'data/Dataset/Subject'
# subject = pd.read_csv(subject_path / 'Subject.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
# obs_path = subject_path/ 'Observation'
# observation = pd.read_csv(obs_path / 'Observation.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
# image = pd.read_csv(obs_path / 'Image/Image.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
# clinic_path = obs_path/ 'Clinical_Records'
# clinic = pd.read_csv(clinic_path / 'Clinical_Records.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
# observation_clinic_asso = pd.read_csv(obs_path / 'Clinical_Records_Observation.csv').drop(
#     columns=['RCT', 'RMT', 'RCB', 'RMB'])
# report = pd.read_csv(obs_path / 'Report/Report.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
# RNFL_OCR = pd.read_csv(obs_path / 'Report/OCR_RNFL/OCR_RNFL.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
# HVF_OCR = pd.read_csv(obs_path / 'Report/OCR_HVF/OCR_HVF.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])

,RID,RCT,RMT,RCB,RMB,Field_Size,Strategy,False_Neg,False_Pos,Fixation_Loss,...,Visual_Acuity,GHT,PSD.1,Abs_Perc,Abs_Val,Pat_Perc,Pat_Val,Raw_Val,Report,Image_Side
0,2-B93Y,2024-05-20 23:15:48.182905+00,2024-10-18 18:55:11.793875+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,24-2,SITA Standard,5%,7%,5/15,...,20/30,Outside Normal Limits,5.19,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-ATX0,Right
1,2-B940,2024-05-20 23:15:48.182905+00,2024-10-18 18:55:11.793875+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,24-2,SITA Fast,Off,0%,,...,,Outside Normal Limits,7.85,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-ATX8,Right
2,2-B942,2024-05-20 23:15:48.182905+00,2024-10-18 18:55:11.793875+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,24-2,SITA Standard,0%,0%,0/15,...,,Outside Normal Limits,5.12,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-ATXG,Right
3,2-B944,2024-05-20 23:15:48.182905+00,2024-10-18 18:55:11.793875+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,24-2,SITA Standard,0%,1%,0/15,...,20/30,Within Normal Limits,,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-ATXM,Right
4,2-B946,2024-05-20 23:15:48.182905+00,2024-10-18 18:55:11.793875+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,24-2,SITA Fast,0%,0%,0/11,...,20/20,Within Normal Limits,,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-ATXR,Right
5,2-BB32,2024-05-20 23:59:49.438927+00,2024-10-18 18:55:13.758981+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,24-2,SITA Standard,0%,8%,,...,20/25,Outside Normal Limits,7.25,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-ATX2,Left
6,2-BB34,2024-05-20 23:59:49.438927+00,2024-10-18 18:55:13.758981+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,24-2,SITA Fast,Off,13%,,...,,Outside Normal Limits,8.73,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-ATXA,Left
7,2-BB36,2024-05-20 23:59:49.438927+00,2024-10-18 18:55:13.758981+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,24-2,SITA Standard,0%,2%,0/15,...,20/40,Borderline,,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-ATXP,Left
8,2-BB38,2024-05-20 23:59:49.438927+00,2024-10-18 18:55:13.758981+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,24-2,SITA Fast,,3%,4/11,...,20/20,Within Normal Limits,2.23,\n \n\n ...,\n \n\n...,\n \n\n ...,\n \n\n...,\n \n\n...,2-ATXT,Left
